In [1]:
import os                                                                       #Rūtos_Ciparytės
import time
import numpy as np
import tensorflow as tf
#from datetime import datetime [time and datetime for 'tensorboard']
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model
#from tensorflow.keras.preprocessing.image import ImageDataGenerator []
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D,ZeroPadding2D

In [2]:
%load_ext tensorboard
from tensorflow.keras.callbacks import TensorBoard

In [3]:
from google.colab import drive
drive.mount('/content/drive') #let's mount the drive

Mounted at /content/drive


In [3]:
cwd = os.getcwd()
print(cwd)
os.listdir(os.getcwd())

/content


['.config', 'drive', 'sample_data']

In [4]:
import gzip
import pickle

X,y = pickle.load(gzip.open('/content/drive/MyDrive/flatland_train.data','rb'))

y[y != 0] -= 2
X = X.reshape(X.shape[0], 50, 50, 1) / 255.0

In [5]:
mob = keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3),classifier_activation="softmax")


# Let's keep those weights
for layer in mob.layers:
    layer.trainable = False

    
# Adding new trainable layers
inputs = keras.layers.Input(shape=(50, 50, 1)) #grayscale
upscale = keras.layers.UpSampling2D((3,3))(inputs) #50->150
padding = keras.layers.ZeroPadding2D((6,6))(upscale) #the site does not like padding :? [well I guess that was the reason for an error]

#con_31 = keras.layers.Conv2D(3, kernel_size=(3, 3), activation='relu')(padding) #these two layer make model even less accurate >;C
#con_2 = keras.layers.Conv2D(2, kernel_size=(3, 3), activation='relu')(con_31)
con_3 = keras.layers.Conv2D(3, kernel_size=(3, 3), activation='relu')(padding)

embedding = mob(con_3)

flatten = keras.layers.Flatten()(embedding)
#the model on it's own reaches great results [by only adding 2 additional layers having parameters]
#dropout = keras.layers.Dropout(0.25)(flatten)
#norma = keras.layers.BatchNormalization(axis=-1)(flatten) welp it just starts guessing....
#dense8 = keras.layers.Dense(8, activation="relu")(dropout)
#norma = keras.layers.BatchNormalization(axis=-1)(dense8)
#output = keras.layers.Dense(16, activation='relu')(norma)
output = keras.layers.Dense(5, activation='softmax')(flatten)

# Build the model
model = keras.models.Model(inputs=inputs, outputs=output)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50, 50, 1)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 150, 150, 1)       0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 162, 162, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 160, 3)       30        
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
flatten (Flatten)            (None, 32000)             0         
_________________________________________________________________
dense (Dense)                (None, 5)                

In [8]:
#NAME = "wazhojam-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adam(3e-4),
              metrics=["accuracy"])

In [ ]:
#X.shape

In [14]:
#X=np.repeat(X,3,axis=-1) / 255.0 #welp wont need it anymore because of 'con_3' layer

In [9]:
loss = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.15) #callbacks=[tensorboard]

Epoch 1/10
266/266 [==============================] - 19s 70ms/step - loss: 0.0513 - accuracy: 0.9828 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 2/10
266/266 [==============================] - 18s 66ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 9.2144e-04 - val_accuracy: 1.0000
Epoch 3/10
266/266 [==============================] - 18s 66ms/step - loss: 4.9408e-04 - accuracy: 1.0000 - val_loss: 5.2153e-04 - val_accuracy: 1.0000
Epoch 4/10
266/266 [==============================] - 17s 66ms/step - loss: 3.0091e-04 - accuracy: 1.0000 - val_loss: 3.6588e-04 - val_accuracy: 1.0000
Epoch 5/10
266/266 [==============================] - 17s 65ms/step - loss: 2.0232e-04 - accuracy: 1.0000 - val_loss: 2.6550e-04 - val_accuracy: 1.0000
Epoch 6/10
266/266 [==============================] - 17s 65ms/step - loss: 1.4564e-04 - accuracy: 1.0000 - val_loss: 2.0224e-04 - val_accuracy: 1.0000
Epoch 7/10
266/266 [==============================] - 17s 66ms/step - loss: 1.0955e-04 - accuracy: 1

In [ ]:
%tensorboard --logdir logs

In [ ]:
#pd.DataFrame(loss.history).plot() minimalistic

In [ ]:
model.save('model.h5')

In [ ]:
from google.colab import files
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''
import gzip
import pickle

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc

model = keras.models.load_model('/content/model.h5')
evaluate('/content/drive/MyDrive/flatland_train.data', model)
'''
#the part of the code which is actually not workig :D but oh well
